## Personas que deberías conocer

El vicepresindente de "fraternización" quiere animar a que hallan más conexiones entre los miembros de la red y te pide que diseñes un "suggester".

El primer instinto es sugerir a los amigos de los amigos. Estos son fáciles de calcular: para cada uno de los amigos del usuario, iterar sobre los amigos de esa persona, y recoger todos los resultados

In [1]:
# Usuarios

users = [
    {"id": 0, "name": "Hero"},
    {"id": 1, "name": "Dunn"},
    {"id": 2, "name": "Sue"},
    {"id": 3, "name": "Chi"},
    {"id": 4, "name": "Thor"},
    {"id": 5, "name": "Clive"},
    {"id": 6, "name": "Hicks"},
    {"id": 7, "name": "Devin"},
    {"id": 8, "name": "Kate"},
    {"id": 9, "name": "Klein"},
]

In [2]:
# Friendships

friendships = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
               (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

In [3]:
for user in users:        # para cada user en users
    user["friends"] = []  # crear propiedad friends como una lista vacía

In [4]:
for i, j in friendships:
    # users[i] es el usuario que su id es i
    users[i]["friends"].append(users[j]) # agregar j como amigo de i
    users[j]["friends"].append(users[i])  # agregar i como amigo de j

In [5]:
def friends_of_friends(user):
    # foaf es el abreviado de "friend of a friend"
    return [
        foaf["id"]
        for friend in user["friends"] # para cada amigo de user
        for foaf in friend["friends"] # obtener sus amigos
    ]

Con este código estamos obteniendo los id de los amigos de los amigos de cada usuario, una vez más se emplea la "manera declarativa" de crear una lista en Python. (Tuve que copiar todo el código de la lista de usuarios y la creacion de la lista de amigos para que funcionara correctamente el código nuevo, el objetivo de estos Notebooks es que sean eso Notebooks donde se puede correr el código y que funcione)

In [6]:
friends_of_friends(users[0])

[0, 2, 3, 0, 1, 3]

Hemos llamado a la función que hemos creado para saber quienes son los amigos de los amigos de Hero (id 0) y nos ha devuelto una lista. La cual contiene el 0 dos veces, ya que de hecho Hero es amigo de sus dos amigos. Incluye ademas los usuarios 1 y 2, a pesar de que ya son amigos de Hero e incluye al usuario Chi (id 3) dos veces, porque Chi es accesible a través de dos amigos diferentes

<img align="center" style="padding:10px;" src="img/fig1.1.png">

In [7]:
print([friend["id"] for friend in users[0]["friends"]])
print([friend["id"] for friend in users[1]["friends"]])
print([friend["id"] for friend in users[2]["friends"]])

[1, 2]
[0, 2, 3]
[0, 1, 3]


Sabiendo que las personas son amigos-de-amigos en múltiples formas, tal vez deberíamos crear un contador para amigos en común. Y sin duda deberíamos crear una función auxiliar para excluir a las personas ya conocidas por el usuario

In [8]:
from collections import Counter

def not_the_same(user, other_user):  # no es el mismo
    """dos usuarios no son el mismo si ambos tienen distintos ids"""
    return user["id"] != other_user["id"]

def not_friends(user, other_user):  # no son amigos
    """other_user no es amigo sino se encuentra en user["friends"] 
    es decir si  no es el mismo para todos los que se encuentran en la lista"""
    
    return all(not_the_same(friend, other_user) for friend in user["friends"])

def friends_of_friends(user):
    return Counter(
        foaf["id"]
        for friend in user["friends"]
        for foaf in friend["friends"]
        if not_the_same(user, foaf)
        and not_friends(user, foaf)
    )

Analisemos rapidamente este codigo solo para las nuevas funciones que han sido incluidas:

El método all() retorna:

    True - si todos los elementos evaluados son True
    False - Si algún elemento evaluado es False

Counter se encuentra en la lib collections por tanto lo importamos, en resumen lo que hace es dado una lista devolvernos un diccionario con los elementos de la lista y su frecuencia, o sea la cantidad de veces que se repiten.

In [9]:
print(friends_of_friends(users[0]))

Counter({3: 2})


In [10]:
print(friends_of_friends(users[3]))

Counter({0: 2, 5: 1})


Como podemos comprobar Chi (id 3) tiene dos amigos en comun con Hero (id 0)

Pero tambien sabemos que podemos agrupar personas por tener intereses similares. Asi que hemos obtenido esa informacion como una lista de <code>(user_id, interest)</code>

In [1]:
interests = [
    (0, "Hadoop"), (0, "Big Data"), (0, "HBase"), (0, "Java"),
    (0, "Spark"), (0, "Storm"), (0, "Cassandra"),
    (1, "NoSQL"), (1, "MongoDB"), (1, "Cassandra"), (1, "HBase"),
    (1, "Postgres"), (2, "Python"), (2, "scikit-learn"), (2, "scipy"),
    (2, "numpy"), (2, "statsmodels"), (2, "pandas"), (3, "R"), (3, "Python"),
    (3, "statistics"), (3, "regression"), (3, "probability"),
    (4, "machine learning"), (4, "regression"), (4, "decision trees"),
    (4, "libsvm"), (5, "Python"), (5, "R"), (5, "Java"), (5, "C++"),
    (5, "Haskell"), (5, "programming languages"), (6, "statistics"),
    (6, "probability"), (6, "mathematics"), (6, "theory"),
    (7, "machine learning"), (7, "scikit-learn"), (7, "Mahout"),
    (7, "neural networks"), (8, "neural networks"), (8, "deep learning"),
    (8, "Big Data"), (8, "artificial intelligence"), (9, "Hadoop"),
    (9, "Java"), (9, "MapReduce"), (9, "Big Data")
]

Por ejemplo Thor (id 4) no tiene amigos en cumun con Devin (id 7) pero comparten un interes por machine learning.

Por tanto resulta facil contruir una funcion que encuentra a los usuarios con un interes determinado.

In [2]:
def who_like(target_interest):
    return [
        user_id
        for user_id, user_interest in interests
        if user_interest == target_interest
    ]

In [3]:
who_like("machine learning")

[4, 7]